In [ ]:
import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import glob, os

import itertools

from backtester.exoinfo import EXOInfo
from exobuilder.data.exostorage import EXOStorage
from IPython.core.display import display, HTML

import statsmodels.tsa.api as smt
import statsmodels.api as sm

import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go

import pandas as pd
import numpy as np
import scipy
import pprint

%pylab inline
figsize(20,10)

In [ ]:
# Loading global setting for MongoDB etc.
from scripts.settings import *

try:
    from scripts.settings_local import *
except:
    pass

storage = EXOStorage(MONGO_CONNSTR, MONGO_EXO_DB)
pp = pprint.PrettyPrinter(indent=4)

exo_list = storage.exo_list()
pp.pprint(exo_list)


df = pd.DataFrame()

for e in exo_list:
    exo_series, exo_dict = storage.load_series(e)
    
    exo_series = exo_series[exo_series.columns[0]]
    df[exo_dict['name']] = exo_series
#exo_series.plot()

In [ ]:
df.tail()

In [ ]:
px_ser = df.CL_PutSpread
px_ser.asfreq('B').dropna().plot()

# Data examination

In [ ]:
#ts = px_ser.asfreq('B').dropna()
ts = px_ser.rolling(10).mean().diff().dropna()
#ts = (px_ser - px_ser.ewm(10).mean()).dropna()
#ts = px_ser.diff(1).dropna()

#ts = (px_ser - px_ser.shift(1)).dropna()
#ts = px_ser.pct_change(1).dropna()
#ts = ((px_ser - px_ser.ewm(10).mean()) / px_ser.ewm(10).std()).dropna()

In [ ]:
#ts.cumsum().ix['2010':].plot()
ts.ix[:'2016'].plot(secondary_y=True)

#px_ser.ix['2010':].plot()
px_ser.ix[:'2016'].plot()

In [ ]:
smt.adfuller(ts)

In [ ]:
sm.graphics.tsa.plot_acf(ts, lags=70)

# Model

In [ ]:
model = smt.ARIMA(ts, order=(4,0,1)).fit()


forecast_period = 10

forecast_daterange = pd.date_range(start=ts.index[-1] + pd.DateOffset(1), periods=forecast_period)

forecast_values = model.forecast(forecast_period)

forecast_confint_df = pd.DataFrame(model.forecast(forecast_period)[2], 
                                   index=forecast_daterange, columns=['Low_ConfInterval_Boundary', 'High_ConfInterval_Boundary'])

forecast_series = pd.Series(forecast_values[0], index=forecast_daterange)


ts.ix['2015':].plot()

forecast_confint_df.Low_ConfInterval_Boundary.ix['2015':].plot()
forecast_confint_df.High_ConfInterval_Boundary.ix['2015':].plot()
#forecast_series.cumsum().plot()
#model_result.fittedvalues.ix['2015':].plot()
#pd.concat([centered_ma, forecast_series]).ix['2015-12':].plot()
forecast_series.ix['2015':].plot()
#model.fittedvalues.ix['2015':].plot()


# Streaming























In [ ]:
#py.sign_in('tmqr', '8rctfyu197')
py.sign_in('dmveden', 'rzbs7xw8ft')

tls.set_credentials_file(username='dmveden', api_key='rzbs7xw8ft')

In [ ]:
stream_ids = tls.get_credentials_file()['stream_ids']
print(stream_ids)

In [ ]:
# Get stream id from stream id list 
stream_id1 = 'twfcm3v44s'
stream_id2 = '5dnfr3e5d8'
stream_id3 = '1ynb4ujpbt'
stream_id4 = '8k95n9xp75'

# Make instance of stream id object 
stream_1 = go.Stream(
    token=stream_id1,  # link stream id to 'token' key
    maxpoints=60      # keep a max of 80 pts on screen
)

# Make instance of stream id object 
stream_2 = go.Stream(
    token=stream_id2,  # link stream id to 'token' key
    maxpoints=60      # keep a max of 80 pts on screen
)

# Make instance of stream id object 
stream_3 = go.Stream(
    token=stream_id3,  # link stream id to 'token' key
    maxpoints=60      # keep a max of 80 pts on screen
)

# Make instance of stream id object 
stream_4 = go.Stream(
    token=stream_id4,  # link stream id to 'token' key
    maxpoints=60      # keep a max of 80 pts on screen
)



trace1 = go.Scatter(x=[], y=[], stream=stream_1, name='Time Series')
trace2 = go.Scatter(x=[], y=[], stream=stream_2, name='Forecast Values', marker=dict(color='rgb(148, 103, 189)'))
trace3 = go.Scatter(x=[], y=[], stream=stream_3, name='High Confidence Interval, 95%', marker=dict(color='rgb(0, 255, 0'))
trace4 = go.Scatter(x=[], y=[], stream=stream_4, name='Low Confidence Interval, 95%', marker=dict(color='rgb(255, 0, 0'))

data = go.Data([trace1, trace2, trace3, trace4])
#data = go.Data([trace1, trace2])

# Add title to layout object
layout = go.Layout(title='ARIMA Calculation Process')

# Make a figure object
fig = go.Figure(data=data, layout=layout)

# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig, filename='python-streaming')

In [ ]:
# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s_1 = py.Stream(stream_id=stream_id1)
s_2 = py.Stream(stream_id=stream_id2)
s_3 = py.Stream(stream_id=stream_id3)
s_4 = py.Stream(stream_id=stream_id4)

# We then open a connection
s_1.open()
s_2.open()
s_3.open()
s_4.open()

# (*) Import module keep track and format current time
import datetime 
import time   
 
i = 100  # a counter

forecast_series = pd.Series()

forecast_confint_df = pd.DataFrame()


append_new_forecasts_flag = True


while True:
    if i >= px_ser.size:
        break
    
    try:
        model = smt.ARIMA(ts.iloc[:i], order=(4,1,0)).fit()
    
    except Exception:
        i += forecast_period
        continue

    forecast_period = 1
    
    forecast_daterange = pd.date_range(start=ts.iloc[:i].index[-1] + pd.DateOffset(1), periods=forecast_period)
    #forecast_daterange = pd.date_range(start=ts.iloc[:i].index[-1] + pd.DateOffset(1), freq='B')
    
    forecast_values = model.forecast(forecast_period)

    if append_new_forecasts_flag == True:
        
        if forecast_confint_df.size == 0:    
            forecast_confint_df = pd.DataFrame(model.forecast(forecast_period)[2], 
                                               index=forecast_daterange, 
                                               columns=['Low_ConfInterval_Boundary', 'High_ConfInterval_Boundary'])
        else:
            forecast_confint_df = pd.concat([forecast_confint_df, pd.DataFrame(model.forecast(forecast_period)[2], 
                                                                               index=forecast_daterange, 
                                                                               columns=['Low_ConfInterval_Boundary', 
                                                                                        'High_ConfInterval_Boundary'])])

        if forecast_series.size == 0:    
            forecast_series = pd.Series(forecast_values[0], index=forecast_daterange)

        else:
            forecast_series = pd.concat([forecast_series, pd.Series(forecast_values[0], index=forecast_daterange)])
    
    
    if append_new_forecasts_flag == False:
        forecast_confint_df = pd.DataFrame(model.forecast(forecast_period)[2], 
                                           index=forecast_daterange, 
                                           columns=['Low_ConfInterval_Boundary', 'High_ConfInterval_Boundary'])

        forecast_series = pd.Series(forecast_values[0], index=forecast_daterange)


    
    # Current time on x-axis, random numbers on y-axis
    x_1 = ts.iloc[:i].index
    y_1 = ts.iloc[:i]
     
    x_2 = forecast_series.iloc[:i].index
    y_2 = forecast_series.iloc[:i] 
    
    x_3 = forecast_confint_df.High_ConfInterval_Boundary.iloc[:i].index
    y_3 = forecast_confint_df.High_ConfInterval_Boundary.iloc[:i] 
    
    x_4 = forecast_confint_df.Low_ConfInterval_Boundary.iloc[:i].index
    y_4 = forecast_confint_df.Low_ConfInterval_Boundary.iloc[:i] 
    
    
    # Send data to your plot
    s_1.write(dict(x=x_1, y=y_1))  

    s_2.write(dict(x=x_2, y=y_2))  
 
    s_3.write(dict(x=x_3, y=y_3))  
 
    s_4.write(dict(x=x_4, y=y_4))  
 
    #     Write numbers to stream to append current data on plot,
    #     write lists to overwrite existing data on plot
    
    time.sleep(0.2)    
    
    x_1 = ts.iloc[:i+forecast_period].index
    y_1 = ts.iloc[:i+forecast_period]
    s_1.write(dict(x=x_1, y=y_1))  
    
        
    time.sleep(0.1)  # plot a point every second    
    i += forecast_period
    
   
# Close the stream when done plotting
s_1.close() 
s_2.close() 
s_3.close() 
s_4.close() 